Date: 22/08/2019

Version: 6.0

Environment: Python 3.7.0 and Anaconda 2019.07 (64-bit)

Libraries used:
* pandas 0.24.2 (for data frame, included in Anaconda Python 3.7) 
* re 2.2.1 (for regular expression, included in Anaconda Python 3.7) 


## 1. Introduction

This assignment requires us to analyze the data in the form a text file (Group010.txt) and extract the required information from the semi-structured data in the text file. The text file consists of several patent grants and related information. Our task is to extract the data and transform the data into the CSV and JSON format with the below stated elements.

1. grant_id: a unique ID for a patent grant consisting of alphanumeric characters.
2. patent_kind: a category to which the patent grant belongs.
3. patent_title: a title given by the inventor to the patent claim.
4. number_of_claims: an integer denoting the number of claims for a given grant.
5. citations_examiner_count: an integer denoting the number of citations made by the examiner for a given patent grant (0 if None)
6. citations_applicant_count: an integer denoting the number of citations made by the applicant for a given patent grant (0 if None)
7. inventors: a list of the patent inventors’ names ([NA] if the value is Null).
8. claims_text: a list of claim texts for the different patent claims ([NA] if the value is Null).
9. abstract: the patent abstract text (‘NA’ if the value is Null)

More details will be provided in the following sections.

# 2. Import Libraries

In [ ]:
import re
import pandas as pd 

# 3. Examining and Loading Data

As a first step, we can read first few lines of the file Group010.txt so that we can examine the given data.

In [ ]:
inputFile = open("Group010.txt", "r", encoding='utf-8')

print('\n'.join([inputFile.readline().strip() for i in range(0, 10)]))

inputFile.close()

We can see that the input text file has an XML declaration <?xml...?> and a root tag <us-patent-grant...>. Based on this information, it is possible to properly delimit an XML document so that required information can be extracted.

A regex is defined so that the strings starting with an XML declaration <?xml..> and ending with the closing tag </us-patent-grant> are captured individually. The non-greedy pattern .*? is necessary so that the whole file is not matched. The regex also uses the pattern [\s\S] (white space or non white space characters) which causes to capture everything, even line breaks, between the XML declaration and the closing tag.

In [ ]:
# read the whole file
inputFile = open("Group010.txt", "r", encoding='utf-8')
text = inputFile.read()
    
# matches everything between the XML declaration and the root closing tag
regex = r'<\?xml[\s\S]*?</us-patent-grant>' 
patents = re.findall(regex, text)
print(len(patents))

inputFile.close()

We found out that we have information for 150 patents.

# 4. Parsing the File and Data extraction

Based on the pattern search for various elements as stated above, we need to extract the required data.
For that, we need to initialize few variables and lists to move ahead with the parsing and extracting of the data.

In [ ]:
citedByApplicant = "applicant"
citedByExaminer = "examiner"
examinerCount = 0
applicantCount = 0
inventorList = []
claimList = []
lineCounter = 1

Next, we read the whole input file and store the data in form of a string.

In [ ]:
inputFile = open("Group010.txt", "r", encoding = 'utf-8')
jsonFile = open('Group010.json','w', encoding = 'utf-8')

lines = inputFile.read()

As we did the pattern matching based on the xml tags earlier, we now isolate data related to a patent and store all related data as an element in the form of a list using findall functionality of regex. 

Below code creates a list of tuples. Each tuple has data related to a particular patent. We pick each element (data related to a patent) and extract the required information.

In [ ]:
patentDataList = re.findall(r'<us-patent-grant((.|\n)*?)</us-patent-grant>',lines)

With the help of pandas library, we create a dataframe object which stores all the columns as stated above and it's related data in tabular form.

In [ ]:
df = pd.DataFrame(columns=['grant_id','patent_title','kind','number_of_claims','inventors','citations_applicant_count','citations_examiner_count','claims_text','abstract'])

The below section of code is where we loop through each index in the patentDataList and extract the required values for their respective columns as stated above. 

### grant_id 
It is a unique id and alphaumeric in nature. We extract it from the <us-patent-grant...> tag. It is extracted in the section which begins with #Grant ID. 

Line from the file : `<us-patent-grant lang="EN" dtd-version="v4.5 2014-04-03" file="`**`US10360351`**`-20190723.XML" status="PRODUCTION" id="us-patent-grant" country="US" date-produced="20190709" date-publ="20190723">`
    
In the above example, the portion between 'file="' and '-20190723.XML"' i.e. 'US10360351' is the grant_id that we need to extract. So, we are using the pattern _'file="(.*?)-'_ and extracting the grant id from it using the group functionality in regex.

### patent_title 
It is a title given by the inventor to the patent claim. From the given input file, we determined that the string enclosed between invention title is the patent title. It is extracted in the section which begins with #Patent Title.

Line from the file : `<invention-title id="d2e51">`**`Authentication translation`**`</invention-title>`

In the above example, the portion between `<invention-title id="d2e51">` and `</invention-title>` i.e. 'Authentication translation' is the patent_title. So, we are using the pattern _'<invention-title(.*?)>(.*?)</invention-title>'_ and extracting the second pattern from it using the group functionality of regex.

### kind 
It is a category to which the patent grant belongs. From the given input file, we determined that the field appl-type in application reference tag is the patent kind. It is extracted in the section which begins with #Patent Kind.

Line from the file : `<application-reference appl-type="`**`utility`**`">`

In the above example, the portion between 'appl-type="' and '">' i.e. 'utility' is the patent_kind. So, we are using the pattern _'appl-type="(.*?)">'_ and extracting the patent kind from it using the group functionality of regex. 
After we get the kind, we are running another if statement to extract the kind of publication it is and based on that we are specifying whether it was published with a particular application or not and also specifying the type of application it is and formatting the final string.
    
### number_of_claims 
It is an integer denoting the number of claims for a given grant. From the given input file, we determined that the number enclosed between number-of-claims tag is the number_of_claims. It is extracted in the section which begins with #Number Of Claims.

Line from the file : `<number-of-claims>`**`10`**`</number-of-claims>`

In the above example, the portion between `<number-of-claims>` and `</number-of-claims>` i.e. '10' is the number_of_claims. So, we are using the pattern _'<number-of-claims>(.*?)</number-of-claims>'_ and extracting the pattern from it using the group functionality of regex.

### inventors 
It is a list of the patent inventors’ names. From the given input file, we determined that the first-name and last-name tags under inventor tag (under the parent inventors tag) contains the name. If there are more than one inventor tag which has last-name and first-name tags, we have to append the names into a list. It is extracted in the section which begins with #Inventors Name List.

Lines from the file : 
`<inventors>`<br>
`<inventor sequence="001" designation="us-only">`<br>
`<addressbook>`<br>
**`<last-name>Jakobsson</last-name>`<br>
`<first-name>Bjorn Markus</first-name>`**<br>
`<address>`<br>
`<city>Portola Valley</city>`<br>
`<state>CA</state>`<br>
`<country>US</country>`<br>
`</address>`<br>
`</addressbook>`<br>
`</inventor>`<br>
`</inventors>`

In the above example, the portion between `<last-name>` and `</last-name>` i.e. 'Jakobsson' gives us the last name of the inventor where as the portion between `<first-name>` and `</first-name>` i.e. 'Bjorn Markus' gives us the first name of the inventor. So, we fetch the entire data between inventors tag using _`<inventors>(.*?)</inventors>`_ pattern and then using findall funtionality of regex, we are finding out the string between the last-name and first name tags. This string has '\n', `</last-name>` tag and `<first-name>` tag which should be removed. Hence, we replace the newline character and split the string based on the tags mentioned in previous line. Since we have last name in first position followed by first name, we have to swap the indices and append it to the inventors list. This is for one inventor. Similarly, for n inventors, we will have n names appended to the inventor list. In order to display the output as desired, we join the names in the inventor list by comma and enclose them between '[]'.



### citations_applicant_count and citations_examiner_count 
citations_examiner_count is an integer denoting the number of citations made by the examiner for a given patent grant where as citations_applicant_count is an integer denoting the number of citations made by the applicant for a given patent grant. From the given input file, we determined that the string enclosed between category tag helps us decide if the citation is made by an examiner or an applicant. We can find the number of citations by counting the number of occurrences of 'cited by examiner' or 'cited by applicant'. It is extracted in the section which begins with #Cited By Applicant or Examiner Count.

Line from the file : `<category>cited by `**`applicant`**`</category>` or `<category>cited by `**`examiner`**`</category>`

In the above example, the string enclosed between category tag contains 'cited by applicant' or 'cited by examiner'. So, we are using the pattern _`<category>cited by (.*?)</category>`_ and counting the number of occurences of each and incrementing their respective variable valuesby one.


### claims_text
It is a list of claim texts for the different patent claims. From the given input file, we determined that the claim text information is enclosed between claims tag. We need to fetch each claim text from a particular claim id within the claims under a patent grant id. It is extracted in the section which begins with #Claim Text.

Lines from the file : <br>
`<claims id="claims">`<br>
`<claim id="CLM-00001" num="00001">`<br>
`<claim-text>`**`I claim the ornamental design for a makeup applicator, substantially as shown and described.`**`</claim-text>`<br>
`</claim>`<br>
`</claims>`

In the above example, we need to extract the string between `<claim-text>` and `</claim-text>`. Once we fetch everything between claims tags, we remove all the xml tags, replace newline character and strip starting/trailing spaces. It is observed from the cleaned data that few records start with serial number whereas few records start with plain text. So, using the findall functionality of regex, we divided the string into a list with serial number or no serial number and plain text. In order to display output as required, we concated the first and second element of the list mentioned earlier, appended it to a list, split it based on comma and finally, enclosing the newly formed string between '[]'


### abstract
It is the patent abstract text. It is extracted in the section which begins with #Abstract.

Line from the file :<br>
`<abstract id="abstract">`<br>
`<p id="p-0001" num="0000">`
**`Authentication translation is disclosed. A request to access a resource is received at an authentication translator, as is an authentication input. The authentication input corresponds to at least one stored record. The stored record is associated at least with the resource. In response to the receiving, a previously stored credential associated with the resource is accessed. The credential is provided to the resource.`**
`</p>`<br>
`</abstract>`

In the above example, the portion between `<abstract id="abstract">` and `</abstract>` gives us the abstract text. Once the string is extracted, we are stripping off the starting/trailing spaces and then we are fetching the data between paragrapgh tags _`<p(.*?)>(.*?)</p>`_. In this pattern, we extract the second group as it contains the abstract text.


After extracting all the required fields we are filling up the dataframe, which we will be later using to create the csv file. 
Then based on certain conditions, mainly the sequence of extracting data from the input file, we are writing data into a file with json extention which in the end becomes our output file in jsonn format.

In [ ]:
for i in range(len(patentDataList)):
    line = patentDataList[i][0]   # Picking index 0 from the list of tuples in patentDataList
    inventorListString = ""       # Initialize the inventorListString (used under inventor name section)
    
    
    # Remove multiple spaces
    line = re.sub('\s\s+', ' ', line)
        
    
    #Grant ID
    grantId = re.search('file="(.*?)-', line)
    grantId = grantId.group(1).strip() 

    
    #Patent Title    
    patTitle = re.search('<invention-title(.*?)>(.*?)</invention-title>', line)
    patTitle = patTitle.group(2).strip()


    #Patent Kind
    patentKind = re.search('appl-type="(.*?)">', line)
    patentKind = patentKind.group(1).strip()
    
    if patentKind == "utility":
        publRef = re.search('<publication-reference>(.*?)</publication-reference>', line, re.DOTALL)
        if publRef is not None:
            publRef = publRef.group(1).strip()
            pubKind = re.search('<kind>(.*?)</kind>', publRef, re.DOTALL)
            pubKind = pubKind.group(1).strip()
            if pubKind == "B2":
                patentKind = "Utility Patent Grant (with a published application) issued on or after January 2, 2001."
            elif pubKind == "B1":
                patentKind = "Utility Patent Grant (no published application) issued on or after January 2, 2001."
            else:
                patentKind = "Utility Patent"
    elif patentKind == "design":
        patentKind = "Design Patent"
    elif patentKind == "reissue":
        patentKind = "Reissue Patent"
        
     
    #Number Of Claims
    numberOfClaims = re.search('<number-of-claims>(.*?)</number-of-claims>', line)
    numberOfClaims = numberOfClaims.group(1).strip()  
    
    
    #Inventors Name List
    inventors = re.search('<inventors>(.*?)</inventors>', line, re.DOTALL)    # Fetch everything between the inventors tag
    if inventors is not None:
        inventors = inventors.group(1).strip()
        inventors = re.findall(r'<last-name>((.|\n)*?)</first-name>',inventors) # Fetch everything between last name and first name tags
        for i in range(len(inventors)):
            inventor = inventors[i][0]
            inventor = inventor.replace("\n", "")    # Replace new lines with nothing
            nameList = inventor.split("</last-name><first-name>")   # Split the string based on '</last-name><first-name>'
            inventorList.append(nameList[1] + " " + nameList[0])   # Concating the first name and last name and adding this concated string(name) into inventorList
            
        inventorListString = (', ').join(inventorList)     # Join each name with a comma
        inventorListString = "[" + inventorListString + "]"    # Enclose the names with []
        inventorList = []    # Setting inventorList to nothing to re-use it for next iteration
    else:
        inventorListString = "[NA]"
    
    
    #Cited By Applicant or Examiner Count
    applicantOrExaminer = re.findall('<category>cited by (.*?)</category>', line)
    for element in applicantOrExaminer:
        if element == citedByExaminer:
            examinerCount += 1            # Increment count by 1 if cited by examiner
        elif element == citedByApplicant:
            applicantCount += 1           # Increment count by 1 if cited by applicant
     
    
    # Claim Text
    claimDetails = re.findall(r'<claims id="claims">((.|\n)*?)</claims>',line)    # Fetch the claims data between claims tag
    claimDetails = claimDetails[0][0]            # Picking index 0 from the list of tuples in patentDataList
    claimDetails = re.sub('<[^<]+>', "",claimDetails)    # Remove every tags within the string
    claimDetails = claimDetails.replace("\n","")       # Replace newline character with nothing in the string
    claimDetails = claimDetails.strip()      # Strip off starting and trailing spaces

    claimDetailsList = re.findall(r'(\d+\.)*(.*?\.)',claimDetails)     # Fetch the cleaned claims data
        
        
    # As the above statement creates list of tuples with first element as serial number/nothing and second element as the statement
    # We concat those elements to form a string and append it to claimList
    for element in claimDetailsList:
        claimList.append(element[0] + element[1])     
    
    # The claimList elements are joined by comma to form a string and enclosed with [] as shown in sample output
    claimText = (',').join(claimList)
    claimText = "[" + claimText + "]"
    claimList = []
    
    if claimText == "[]":
        claimText = "[NA]"  
    
    
    # Abstract
    abstract = re.search('<abstract id="abstract">(.*?)</abstract>', line, re.DOTALL)    # Fetch abstract information
    if abstract is not None:
        abstract = abstract.group(1).strip()
        abstract = abstract.replace("\n","")
        abstract = abstract.strip()
        abstract = re.sub('<[^<]+>', "",abstract) # Remove tags
        abstract = abstract.strip()
    else:
        abstract = "NA"    
    
    # Append the extracted data for each patent into dataframe object
    df = df.append({'grant_id': grantId, 'patent_title': patTitle, 'kind': patentKind, 'number_of_claims': int(numberOfClaims), 'inventors': inventorListString, 'citations_applicant_count': applicantCount, 'citations_examiner_count': examinerCount, 'claims_text': claimText, 'abstract': abstract}, ignore_index=True)
       
    # Forming the json string for a particular patent
    if lineCounter == 1:
        jsonString = '{"' + grantId + '":' + '{"patent_title":' + '"' + patTitle + '",' + '"kind":' + '"' + patentKind + '",' \
                    '"number_of_claims":' + str(numberOfClaims) + ',' + '"inventors":' + '"' + inventorListString + '",' + '"citations_applicant_count":' + str(applicantCount) + ',' \
                    '"citations_examiner_count":' + str(examinerCount) + ',' + '"claims_text":' + '"' + claimText + '",' + '"abstract":' + '"' + abstract + '"},'
        
    elif lineCounter >= 2 and lineCounter <= (len(patentDataList)-1):
        jsonString = '"' + grantId + '":' + '{"patent_title":' + '"' + patTitle + '",' + '"kind":' + '"' + patentKind + '",' \
                    '"number_of_claims":' + str(numberOfClaims) + ',' + '"inventors":' + '"' + inventorListString + '",' + '"citations_applicant_count":' + str(applicantCount) + ',' \
                    '"citations_examiner_count":' + str(examinerCount) + ',' + '"claims_text":' + '"' + claimText + '",' + '"abstract":' + '"' + abstract + '"},'
        
    elif lineCounter == (len(patentDataList)):
        jsonString = '"' + grantId + '":' + '{"patent_title":' + '"' + patTitle + '",' + '"kind":' + '"' + patentKind + '",' \
                    '"number_of_claims":' + str(numberOfClaims) + ',' + '"inventors":' + '"' + inventorListString + '",' + '"citations_applicant_count":' + str(applicantCount) + ',' \
                    '"citations_examiner_count":' + str(examinerCount) + ',' + '"claims_text":' + '"' + claimText + '",' + '"abstract":' + '"' + abstract + '"}}'
    

    # Writing into the json file
    jsonFile.write(jsonString)  
    lineCounter += 1
        
    # Setting the counts to zero so that for the next iteration, it'll start from zero
    examinerCount = 0
    applicantCount = 0

The above code extracts the required information from the input txt file and appends that information into dataframe object.

Below code is to export the extracted information present in dataframe object to a CSV file without the default index in dataframe.

In [ ]:
df.to_csv('Group010.csv', index = False, encoding='utf-8')

Finally, once the data is parsed and extracted, we are closing the json file and the input file.

In [ ]:
jsonFile.close()
inputFile.close()